<a href="https://colab.research.google.com/github/EvanWAppel/work-examples/blob/main/Time_Discrepancy_Analysis_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Time Discrepancy Analysis Engine

This Python notebook is designed to compile and analyze several sets of data and to produce tools for discovering discrepancies.

When run, the engine will produce a set of excel files with findings.

## Step 1:

Upload data to content folder in filesystem.

Files need to be named thusly, followed by the period of time:

1.   employee_file, N/A
2.   exceptions, First day of last pay period to yesterday.
3.   schedule, First day of last pay period to last day of next pay period.
4.   project, First day of last pay period to yesterday.
5.   availability, First day of last pay period to last day of next pay period.
6.   scores, First day of last pay period to yesterday.
7.   Variance, Past 30 days.
8.   sessions, First day of last pay period to yesterday.
9.   slices, Past seven days.

## Step 2:

Run the Engine. Click "Runtime" in the toolbar. Then click "Run All."

## Step 3: 

Download the file.

## More information?

For more information, please see the following cell which details specifications. If you can't find what you're looking for, please call Evan Appel at 702-466-4498.





#Query References
##Sessions Query

```

 /*TITLE: Agent Sessions Query
   AUTHOR: Evan Appel
   UPDATED: May 7, 2020*/
 
        SELECT LOWER(a.FirstName + ' ' + a.LastName) AS agent
             , LEFT(dirsrv.text,PATINDEX('% %',dirsrv.text)) AS bamboo_id
             , o2.Name AS project_name
             -- The following sets the login and logout times to ISO 8601 for coding purposes
             /*, FORMAT(s.loginDateTime,'yyyy-MM-ddTHH:mm:ss.fffZ') AS LOGIN
             , FORMAT(s.logoutDateTime, 'yyyy-MM-ddTHH:mm:ss.fffZ') AS logout */
             , CAST(DATEADD(ss,DATEDIFF(ss,GETUTCDATE(), GETDATE()),s.loginDateTime) AS DATE) AS LOGIN_DATE
             , CAST(DATEADD(ss,DATEDIFF(ss,GETUTCDATE(), GETDATE()),s.loginDateTime) AS TIME) AS lOGIN_TIME
             , CAST(DATEADD(ss,DATEDIFF(ss,GETUTCDATE(), GETDATE()),s.logoutDateTime) AS DATE) AS LOGOUT_DATE
             , CAST(DATEADD(ss,DATEDIFF(ss,GETUTCDATE(), GETDATE()),s.logoutDateTime) AS TIME) AS LOGOUT_TIME
             , DATEDIFF(ms, s.loginDateTime, s.logoutDateTime)/1000 AS sesh_dur
             , [talkTime]
             , [waitingTime]
             , [pauseTime]
             , [reviewTime]
             , [talkTime] + [waitingTime] + [pauseTime] + [reviewTime] AS soat 
            -- , ap.total AS completes
             --, (((DATEDIFF(ms, s.loginDateTime, s.logoutDateTime)/1000)  / ap.total) / 60) AS prod_rate
           /* */
          FROM [VoxcoSystem].[dbo].[AgentSession] s
     LEFT JOIN [VoxcoSystem].[dbo].[tblAgents] a 
            ON s.userId = a.k_Id
     LEFT JOIN [VoxcoSystem].[dbo].[tblObjects] o 
            ON s.projectId = o.k_Id
     LEFT JOIN [VoxcoSystem].[dbo].[tblObjects] o2 
            ON o.ParentId = o2.k_Id
     LEFT JOIN [VoxcoSystem].[dbo].[History_DirSrvObjName] dirsrv
            ON s.userId = dirsrv.id
     LEFT JOIN [VoxcoSystem].[dbo].[AgentSessionConnectionTimes] c 
            ON s.sessionId = c.sessionId
     --LEFT JOIN (SELECT * FROM [VoxcoSystem].[dbo].[AgentSessionProductivity]
               --  WHERE resultCode = 'CO') ap
           -- ON s.sessionId = ap.sessionId
            -- Date parameter set to yesterday, local time.
         WHERE CAST(DATEADD(ss,DATEDIFF(ss,GETUTCDATE(), GETDATE()),s.loginDateTime) AS DATE) BETWEEN '2021-08-28' AND '2021-09-03' -- weekend code
        -- CAST(DATEADD(ss,DATEDIFF(ss,GETUTCDATE(), GETDATE()),s.loginDateTime) AS DATE) = CAST(getdate() - 1 AS DATE) 
          -- and a.FirstName = 'richard'
      ORDER BY loginDateTime DESC
```
##Slices Query
Use in Cloudwatch, prod event bus

```
fields time, detail.associate, detail.start, detail.end, detail.minutes, detail.punch, detail.project, `detail.slice-type`
| filter (`detail-type` = "slice") 
and `detail.slice-type` not like "epoch" 
and `detail.punch` not in ["out","manager","courtesy-patrol","monitoring","team-lead-assistant","office-maintenance","office-technician","office-admin","trainer","quota-managemeent",""]
and strlen(`detail.punch`) >=1
and `detail.project` not like 'nis'
| sort @timestamp desc
|limit 10000
```




# Imports

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
from datetime import date
import pytz 
from dateutil import parser
import numpy as np
from google.colab import files
%load_ext google.colab.data_table
from google.colab import data_table


# Options and Settings

In [ ]:
#Sets options so the data frame doesn't break into "pages"
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 800)
# When I assign a column to itself to change its datatype to datetime it throws a warning, this suppresses that warning. #### USE EXTREME CAUTION
pd.set_option('mode.chained_assignment', None)

# Ingesters

In [ ]:
# Employee Data
try:
  employeeFile = pd.read_csv('employee_file.csv',parse_dates=True)
except:
  print("No Employee File Available")
else:
  print("Employee File checker ...")
  checker = 0
  if len(employeeFile) > 0:
    print("Employee File is non-empty. "+str(len(employeeFile))+" records found.")
  else:
    print("Employee File problem: Empty File! "+str(len(employeeFile))+" records found.")
  for i in employeeFile.columns :
    if i in ['Last name, First name', 'Employee #', 'Status', 'Division', 'Job Title','Hire Date']:
      checker += 1
  if checker == 6:
    print("Required columns present for Employee File")
  else:
    print("Check Employee File document for correct fields, referenced above.")

# Exception Data
try:
  exceptions = pd.read_csv('exceptions.csv',parse_dates=True)
except:
  print("No exceptions file Available")
else:
  print("Exceptions file checker ...")
  if len(exceptions) > 0:
    print("Employee File is non-empty. "+str(len(exceptions))+" reecords found.")
  else:
    print("Employee File is empty! "+str(len(exceptions))+" records found.")
  print("Data begins on "+str(exceptions['date'].min())+"\n"+"And ends on "+str(exceptions['date'].max()))

# Schedule Data
try:
  schedule = pd.read_csv('schedule.csv')
except:
  print("No schedule file Available")
else:
  print("Schedule file checker ...")
  if len(schedule) > 0:
    print("Schedule File is non-empty. "+str(len(schedule))+" reecords found.")
  else:
    print("Schedule File is empty! "+str(len(schedule))+" records found.")
  print("Data begins on "+str(schedule['Date'].min())+"\n"+"And ends on "+str(schedule['Date'].max()))
# Sessions Data
try:
  sessions = pd.read_excel('sessions.xlsx', parse_dates=True)
except:
  print("No sessions file Available")
else:
  print("Sessions file checker ...")
  if len(sessions) > 0:
    print("Sessions File is non-empty. "+str(len(sessions))+" records found.")
  else:
    print("Sessions File is empty! "+str(len(sessions))+" records found.")
  print("Data begins on "+str(sessions['LOGIN_DATE'].min())+"\n"+"And ends on "+str(sessions['LOGIN_DATE'].max()))
# Slices Data
try:
  slices = pd.read_csv('slices.csv', parse_dates=True)
except:
  print("No slices file Available")
else:
  print("Slices file checker ...")
  if len(slices) > 0:
    print("Slices File is non-empty. "+str(len(slices))+" records found.")
  else:
    print("Slices File is empty! "+str(len(schedule))+" records found.")
  print("Data begins on "+str(slices['detail.start'].min())+"\n"+"And ends on "+str(slices['detail.start'].max()))
# Project Data
try:
  project = pd.read_csv('project.csv',parse_dates=True)
except:
  print("no project file available")
else:
  print("Project file checker...")
  if (len(project)>0):
    print("Project file is non-empty. "+str(len(project))+" records found.")
# Variance Data
try: 
  variance = pd.read_csv('Variance.csv',parse_dates=True)
except:
  print("no variance file available")
else:
  print("variance file checker...")
  if len(variance)>0:
    print("Variance file is non-empty. "+str(len(variance))+" records found.")
# availability Data
try: 
  availability = pd.read_csv('availability.csv',parse_dates=True)
except:
  print("no availability file available")
else:
  print("availability file checker...")
  if len(availability)>0:
    print("availability file is non-empty. "+str(len(availability))+" records found.")
# Scores data
try: 
  scores = pd.read_csv('scores.csv',parse_dates=True)
except:
  print("no scores file available")
else:
  print("scores file checker...")
  if len(scores)>0:
    print("scores file is non-empty. "+str(len(scores))+" records found.")
# Requested days off
try: 
  requests = pd.read_csv('requests.csv',parse_dates=True)
except:
  print("no requests file available")
else:
  print("requests file checker...")
  if len(scores)>0:
    print("requests file is non-empty. "+str(len(requests))+" records found.")


No Employee File Available
No exceptions file Available
No schedule file Available
No sessions file Available
No slices file Available
no project file available
no variance file available
no availability file available
no scores file available
no requests file available


In [ ]:
e = pd.read_csv('exceptions.csv')
print(e.head())

                                                  id     eid               name  supervisor                      created     date assignee associate-status approval-status  approver approvalTS resolved-status  resolver resolvedTS       project commenters
0  exception:26010_2022-02-20:9be56f2f1fc64528bae...   26010      Lisa Kuntzman     35400.0  2022-02-20T22:03:17.087303Z  2/20/22    35400         APPROVED            NONE       NaN        NaN            NONE       NaN        NaN       s2r1001        NaN
1  exception:105829_2022-02-20:b8671eac74624451bd...  105829      Latanya Mason     34688.0  2022-02-21T10:41:33.476026Z  2/20/22   105829             NONE            NONE       NaN        NaN            NONE       NaN        NaN  cf1009w-cell        NaN
2  exception:105691_2022-02-20:a74a31b61410402bbf...  105691    Scott Lefkowitz     34688.0  2022-02-20T19:47:23.847004Z  2/20/22    34688         APPROVED            NONE       NaN        NaN            NONE       NaN        NaN  cf10

# Table Preparation
This part is to get table data types arranged properly.

In [ ]:
### Employee File
# Only Relevant Columns
employee = employeeFile[["Last name, First name","Employee #","Status","Division","Job Title","Hire Date"]]
# Better column names
employee = employee.rename(columns={"Last name, First name":"full_name",
                                    "Employee #": "eid",
                                    "Hire Date": "hire_date"})
# Set hire date to proper date type
employee["hire_date"] = pd.to_datetime(employee["hire_date"])

NameError: ignored

In [ ]:
### Exceptions
# Only Relevant columns
excep = exceptions[["eid","date","project","id"]]
# Set Date to proper type
excep["date"] = pd.to_datetime(excep["date"])
# Aggregate
excep = excep.groupby(["eid","date","project"], as_index=False)["id"].size().fillna(0)
excep = excep.rename(columns={"size":"exception_count"})
excep = excep[["eid","date","exception_count"]]

NameError: ignored

In [ ]:
### Schedule
# Only Relevant Columns
sched = schedule[["EID","Project Name","Date","StartTime","EndTime"]]
# column names are to be descriptive, undercase and words separated by underscores
sched = sched.rename(columns={"EID":"eid",
                              "Project Name":"sched_project",
                              "Date":"date",
                              "StartTime":"sched_start",
                              "EndTime":"sched_end"})
# Convert the datetimes to proper types. WARNING: Had to disable a warning for the following 3 rows. Note in above section.
sched["sched_start"] = pd.to_datetime(sched["date"] + " " + sched["sched_start"])
sched["sched_end"] = pd.to_datetime(sched["date"] + " " + sched["sched_end"])
sched["date"] = pd.to_datetime(sched["date"])

NameError: ignored

In [ ]:
### Sessions
# Relevant Columns
sesh = sessions[["bamboo_id","project_name","LOGIN_DATE","lOGIN_TIME","LOGOUT_DATE","LOGOUT_TIME","sesh_dur","talkTime","waitingTime","pauseTime","reviewTime","soat"]]
# Rename Columns
sesh = sesh.rename(columns={"bamboo_id":"eid",
                            "project_name":"vc_project",
                            "talkTime":"talk_time",
                            "waitingTime":"waiting_time",
                            "pauseTime":"pause_time",
                            "reviewTime":"review_time"})
# Correct Datatypes
# To get the dates working: basically, the date and time need to be converted to strings and then concatenated, from there, a UTC suffix can be added with the first tz_localize, converted to Pacific time, and then remove the suffix with the second tz_localize
sesh["vc_login"] = pd.to_datetime(sesh["LOGIN_DATE"].astype(str) + " " + sesh['lOGIN_TIME'].astype(str)).dt.tz_localize("US/Pacific").dt.tz_localize(None)
sesh["LOGOUT_DATE"] = sesh["LOGOUT_DATE"].fillna("2021-10-14")
sesh['LOGOUT_TIME'] = sesh['LOGOUT_TIME'].fillna("10:50:00")
sesh["vc_logout"] = pd.to_datetime(sesh["LOGOUT_DATE"].astype(str) + " " + sesh['LOGOUT_TIME'].astype(str)).dt.tz_localize("US/Pacific").dt.tz_localize(None)
sesh["date"] = sesh["vc_login"].dt.date
sesh["eid"] = sesh["eid"].fillna(0)
sesh = sesh[sesh["eid"] != '26045B']
sesh = sesh[sesh["eid"] != 'jleejoyce']
sesh = sesh[sesh["eid"] != 'kbigelow']
sesh = sesh[sesh["eid"] != 'tstrauss']
sesh["eid"] = sesh["eid"].astype("str").astype("int64")

NameError: ignored

In [ ]:
### Slices
# Relevant Columns
sl = slices[["time","detail.associate","detail.start","detail.end","detail.minutes","detail.punch","detail.project"]]
# Rename Columns
sl = sl.rename(columns={"time":"timestamp",
                        "detail.associate":"eid",
                        "detail.start":"ontime_start",
                        "detail.end":"ontime_end",
                        "detail.minutes":"ontime_duration",
                        "detail.punch":"punch",
                        "detail.project":"ontime_project"})
# Data Type Correction
sl["eid"] = sl["eid"].astype("int64")
sl["ontime_start"] = pd.to_datetime(sl["ontime_start"]).dt.tz_convert("US/Pacific").dt.tz_localize(None)
sl["ontime_end"] = pd.to_datetime(sl["ontime_end"]).dt.tz_convert("US/Pacific").dt.tz_localize(None)
sl["date"] = sl["ontime_start"].dt.date
sl["timestamp"] = pd.to_datetime(sl["timestamp"])
# Data Cleaning
# Rows with NAN in certain columns indicate a correction that hasn't been filtered by Cloudwatch
sl = sl.dropna()
# removing corrections
# The problem is that there are duplicates here. Records that have the same start or end times. 
# The younger one is the correct one.
# so, assign them a rank where we look at each start or end time and start counting up backwards
# if there's only one record with that timestamp then it gets a one
# if there's a duplicate start time then the program looks at the timestamp and sees which one is older.
# The oldeest timestamp gets the 1, which ensures when we filter only the rows with a rank of 1
# we will have the latest slices and no corrections.
# It occurs to me that the rank sections should probably group by eid too to avoid 
# Duplicate timestamps between people, but the precision on these timestamps is such that I think it
# Very unlikely that anything but a system operation would end up producing duplicate timestamps
cs = sl[["timestamp",
         "eid",
         "ontime_start",
         "ontime_end",
         "ontime_duration",
         "punch",
         "ontime_project",
         "date"]]
ce = sl[["timestamp",
         "eid",
         "ontime_start",
         "ontime_end",
         "ontime_duration",
         "punch",
         "ontime_project",
         "date"]]

cs =(
        cs.assign(
            rnk=cs.groupby(["ontime_start"])["timestamp"].rank(
                method="first", ascending=False
            )
        )
    )
ce =(
        ce.assign(
            rnk=ce.groupby(["ontime_start"])["timestamp"].rank(
                method="first", ascending=False
            )
        )
    )

sl = pd.concat([cs,ce])
sl = sl[sl["rnk"]==1]
sl = sl.drop_duplicates()
sl["timestamp"] = pd.to_datetime(sl["timestamp"]).dt.tz_localize(None)

NameError: ignored

In [ ]:
### Project
# For total Ontime minutes in metrics
project["date"] = pd.to_datetime(project["date"])
ontMin = project.groupby(["EID","date"],as_index=False).agg({"hours":sum})
ontMin = ontMin.assign(ontimeDuration=ontMin["hours"]*60)

ontMin = ontMin[["EID","date","ontimeDuration"]]
ontMin = ontMin.rename(columns={"EID":"eid",
                                "ontimeDuration": "ontime_duration"})

NameError: ignored

In [ ]:
# availability
availability["date"] = pd.to_datetime(availability["date"]).dt.date
availability["avail_start"] = pd.to_datetime(availability["date"].astype("str")+" "+availability["start"])
availability["avail_end"] = pd.to_datetime(availability["date"].astype("str")+" "+availability["end"])
availability["date"] = pd.to_datetime(availability["date"])
avail = availability[["date","eid","avail_start","avail_end"]]

NameError: ignored

In [ ]:
# variance
excuses = variance[["EID","date","absent"]]
excuses = excuses.dropna()
excuses = excuses.rename(columns={"EID":"eid",
                                  "absent":"absence_code"})
excuses["date"] = pd.to_datetime(excuses["date"])

In [ ]:
# scores
scores = scores.rename(columns={"EID":"eid",
                                "Date":"date",
                                "Score":"score",
                                "Cases":"cases"})
sc = scores.groupby(["eid","date"], as_index=False).agg({"score":np.mean,"cases":np.size})
sc["date"] = pd.to_datetime(sc["date"])
sc["score"] = round(sc["score"].astype("float64"),1)

In [ ]:
# Completion Data
comp = sessions.rename(columns={"bamboo_id":"eid",
                            "project_name":"project"})
# Correct Datatypes
# To get the dates working: basically, the date and time need to be converted to strings and then concatenated, from there, a UTC suffix can be added with the first tz_localize, converted to Pacific time, and then remove the suffix with the second tz_localize
comp["vc_login"] = pd.to_datetime(comp["LOGIN_DATE"].astype(str) + " " + comp['lOGIN_TIME'].astype(str)).dt.tz_localize("US/Pacific").dt.tz_localize(None)
comp["LOGOUT_DATE"] = comp["LOGOUT_DATE"].fillna("2021-10-14")
comp['LOGOUT_TIME'] = comp['LOGOUT_TIME'].fillna("10:50:00")
comp["vc_logout"] = pd.to_datetime(comp["LOGOUT_DATE"].astype(str) + " " + comp['LOGOUT_TIME'].astype(str)).dt.tz_localize("US/Pacific").dt.tz_localize(None)
comp["date"] = comp["vc_login"].dt.date
comp["eid"] = comp["eid"].fillna(0)
#comp = comp[comp["eid"] != '26045B']
#comp = comp[comp["eid"] != 'jleejoyce']
#comp = comp[comp["eid"] != 'kbigelow']
#comp = comp[comp["eid"] != 'tstrauss']
#comp["eid"] = comp["eid"].astype("str").astype("int64")
#comp = comp.groupby(["eid","project","date"],as_index=False).agg({"completes":sum,
#                                                                "calls":sum})
comp["date"] = pd.to_datetime(comp["date"])
comp["project"] = comp["project"].str.replace('_','-').str.lower()

# Metrics

In [ ]:
################################################################### Schedule
# The entire table is based on who should be here, which is determined by the schedule
metrics = sched[["eid","sched_project","date","sched_start","sched_end"]]

################################################################### Employee
# Add employee information
metrics = pd.merge(metrics,employee,on=["eid"],how="left")

################################################################### Exceptions
# Add in how many exceptions they filed
metrics = pd.merge(metrics,excep,on=["eid","date"],how="left").fillna(0)

################################################################### Voxco
# First Voxco Login
# Figure out the first voxco login by person/date
firstVC = sesh.groupby(["eid","date"],as_index=False).agg({"vc_login":min})
# prepare the list of projects
vcProjectIn = sesh[["eid","date","vc_login","vc_project" ]]
# rename project
vcProjectIn = vcProjectIn.rename(columns={"vc_project":"vc_login_project"})
# Join the project to the first login of the day so you only have the project for the first login
firstVC = pd.merge(firstVC,vcProjectIn,on=["eid","date","vc_login"],how="left")
# convert firstVC date to proper format
firstVC["date"] = pd.to_datetime(firstVC["date"])
# Join first login info
metrics = pd.merge(metrics,firstVC,on=["eid","date"],how="left")

# Last Voxco Logout
# Last logout 
lastVC = sesh.groupby(["eid","date"],as_index=False).agg({"vc_logout":max})
# Last logout project
vcProjectOut = sesh[["eid","date","vc_logout","vc_project" ]]
# rename project
vcProjectOut = vcProjectOut.rename(columns={"vc_project":"vc_logout_project"})
# Join project to logout
lastVC = pd.merge(lastVC,vcProjectOut,on=["eid","date","vc_logout"],how="left")
# convert lastVC date to proper format
lastVC["date"] = pd.to_datetime(lastVC["date"])
# Join last logout info
metrics = pd.merge(metrics,lastVC,on=["eid","date"],how="left")

# Voxco Durations
# sum up durations, project agnostic to fit the table
vcTimes = sesh.groupby(["eid","date"],as_index=False).agg({"sesh_dur":sum,
                                                           "soat":sum,
                                                           "talk_time":sum,
                                                           "waiting_time":sum,
                                                           "pause_time":sum,
                                                           "review_time":sum})
# Convert vcTimes date to correct format
vcTimes["date"] = pd.to_datetime(vcTimes["date"])
# Join durations
metrics = pd.merge(metrics,vcTimes,on=["eid","date"],how="left")

# Voxco Count
# count the sessions
vcCount = sesh.groupby(["eid","date"],as_index=False)["eid"].size()
# Convert vcCount date to proper format
vcCount["date"] = pd.to_datetime(vcCount["date"])
vcCount = vcCount.rename(columns={"size":"vc_session_count"})
# Join count
metrics = pd.merge(metrics,vcCount,on=["eid","date"],how="left")

################################################################### Slices
# First OnTime Punch
# Figure out the first OnTime Punch by person/date
firstOT = sl.groupby(["eid","date"],as_index=False).agg({"ontime_start":min})
# prepare the list of projects
otProjectIn = sl[["eid","date","ontime_start","ontime_project" ]]
# rename project
otProjectIn = otProjectIn.rename(columns={"ontime_project":"ontime_start_project"})
# Join the project to the first punch of the day so you only have the project for the first punch
firstOT = pd.merge(firstOT,otProjectIn,on=["eid","date","ontime_start"],how="left")
# convert firstVC date to proper format
firstOT["date"] = pd.to_datetime(firstOT["date"])
# Join first login info
metrics = pd.merge(metrics,firstOT,on=["eid","date"],how="left")

# Last OnTime Punch
# Figure out the last OnTime Punch by person/date
lastOT = sl.groupby(["eid","date"],as_index=False).agg({"ontime_end":max})
# prepare the list of projects
otProjectOut = sl[["eid","date","ontime_end","ontime_project" ]]
# rename project
otProjectOut = otProjectOut.rename(columns={"ontime_project":"ontime_end_project"})
# Join the project to the first punch of the day so you only have the project for the first punch
lastOT = pd.merge(lastOT,otProjectOut,on=["eid","date","ontime_end"],how="left")
# convert firstVC date to proper format
lastOT["date"] = pd.to_datetime(lastOT["date"])
# Join first login info
metrics = pd.merge(metrics,lastOT,on=["eid","date"],how="left")

# Ontime Durations
## This is unreliable because the list of ontime slices may include defunct corrected slices
## Usze the project table instead
# sum up durations, project agnostic to fit the table
#otTimes = sl.groupby(["eid","date"],as_index=False).agg({"ontime_duration":sum})
# Convert otTimes date to correct format
#otTimes["date"] = pd.to_datetime(otTimes["date"])
# Join durations
#metrics = pd.merge(metrics,otTimes,on=["eid","date"],how="left")
metrics = pd.merge(metrics,ontMin,on=["eid","date"],how="left")

# OT Count
# count the sessions
otCount = sl.groupby(["eid","date"],as_index=False)["eid"].size()
# Convert otCount date to proper format
otCount["date"] = pd.to_datetime(otCount["date"])
otCount = otCount.rename(columns={"size":"ontime_slice_count"})
# Join count
metrics = pd.merge(metrics,otCount,on=["eid","date"],how="left")

# OnTime Breaks
breakData = sl[sl["punch"]=="break"]
breakCount = breakData.groupby(["eid","date"],as_index=False).size()
breakSum = breakData.groupby(["eid","date"],as_index=False).agg({"ontime_duration":sum})
breakData = pd.merge(breakCount,breakSum,on=["eid","date"],how="left")
breakData = breakData.rename(columns={"ontime_duration":"break_duration",
                                      "size":"break_count"})
breakData["date"] = pd.to_datetime(breakData["date"])
metrics = pd.merge(metrics,breakData,on=["eid","date"],how="left")

################################################################### Availability
metrics = pd.merge(metrics,avail,on=["date","eid"],how="left")

################################################################### Excuses
metrics = pd.merge(metrics,excuses,on=["date","eid"],how="left")

################################################################### Monitoring
metrics = pd.merge(metrics,sc,on=["date","eid"],how="left")

################################################################### MOAR COLUMNS!!!
# Diff between schedule start and ontime start
metrics = metrics.assign(schedule_to_ontime=metrics["ontime_start"] - metrics["sched_start"])
metrics["schedule_to_ontime"] = metrics["schedule_to_ontime"].dt.total_seconds()
metrics["schedule_to_ontime"] = metrics["schedule_to_ontime"] / 60
# Diff between ontime start and voxco start
metrics = metrics.assign(ontime_to_voxco=metrics["vc_login"] - metrics["ontime_start"])
metrics["ontime_to_voxco"] = metrics["ontime_to_voxco"].dt.total_seconds()
metrics["ontime_to_voxco"] = metrics["ontime_to_voxco"]/60
# Diff between voxco end and ontime end
metrics = metrics.assign(voxco_to_ontime=metrics["ontime_end"] - metrics["vc_logout"])
metrics["voxco_to_ontime"] = metrics["voxco_to_ontime"].dt.total_seconds()
metrics["voxco_to_ontime"] = metrics["voxco_to_ontime"]/60
# Diff between ontime end and schedule end
metrics = metrics.assign(ontime_to_schedule=metrics["sched_end"] - metrics["ontime_end"])
metrics["ontime_to_schedule"] = metrics["ontime_to_schedule"].dt.total_seconds()
metrics["ontime_to_schedule"] = metrics["ontime_to_schedule"] /60

# "sesh_dur" to mins
metrics["sesh_dur"] = metrics["sesh_dur"]/60
# "soat" to mins
metrics["soat"] = metrics["soat"]/60
# "talk_time" to mins
metrics["talk_time"] = metrics["talk_time"]/60
# "waiting_time" to mins
metrics["waiting_time"] = metrics["waiting_time"]/60
# "pause_time" to mins
metrics["pause_time"] = metrics["pause_time"]/60
# "review_time" to mins
metrics["review_time"] = metrics["review_time"]/60
# "schedule_duration" to mins
metrics = metrics.assign(schedule_duration=metrics["sched_end"] - metrics["sched_start"])
metrics["schedule_duration"] = metrics["schedule_duration"].dt.total_seconds()
metrics["schedule_duration"] = metrics["schedule_duration"] / 60
# Work Rate
metrics = metrics.assign(work_rate=metrics["ontime_duration"] / metrics["schedule_duration"])
# Absence Rate
metrics = metrics.assign(absence_rate=metrics["ontime_duration"] / (metrics["schedule_duration"])-1)
# Potential Discrepancy
metrics = metrics.assign(potential_discrepancy=metrics["schedule_duration"] - metrics["sesh_dur"])


################################################################### Reorder table
metrics = metrics[["full_name",
                   "eid",
                   "Job Title",
                   "Division",
                   "Status",
                   "hire_date",
                   "date",
                   "potential_discrepancy",
                   "work_rate",
                   "absence_rate",
                   "absence_code",
                   "exception_count",
                   "score",
                   "cases",
                   "avail_start",
                   "sched_start",
                   "sched_project",
                   "schedule_to_ontime",
                   "ontime_start",
                   "ontime_start_project",
                   "ontime_to_voxco",
                   "vc_login",
                   "vc_login_project",
                   "vc_logout",
                   "vc_logout_project",
                   "voxco_to_ontime",
                   "ontime_end",
                   "ontime_end_project",
                   "ontime_to_schedule",
                   "sched_end",
                   "avail_end",
                   "sched_project",
                   "talk_time",
                   "waiting_time",
                   "pause_time",
                   "review_time",
                   "sesh_dur",
                   "soat",
                   "ontime_duration",
                   "schedule_duration",
                   "vc_session_count",
                   "ontime_slice_count",
                   "break_duration",
                   "break_count"]]

# Pre-filter the list. Only RAs, only CRD, no Staff, no trainees

#print(metrics.dtypes)
m = metrics[(metrics["Job Title"]=="Research Associate")&(metrics["Division"]=="CRD")]
#data_table.DataTable(metrics)
#data_table.DataTable(m)

# Concern List

In [ ]:
# This code block produces a list of concerning circumstances for Workforce Management to address
# The first part establishes some parameters for the concerns list
# The second part creates the list




#Parameters
#How many exceptions trigger an event?
exception_trigger = 2
# What defines a concerning discrepancy?
## Greater than the absolute value of one standard deviation from the mean?
discrepancy_trigger = 30
# Work Rate
work_rate_trigger = 0.7
# Schedule to Ontime
sched_to_ontime_trigger = 5
# Lunch Length
lunch_length = 30
# Lunch Count per 8 hours
lunch_count = 1
# Break Length
break_length = 30
# Break Count
break_count = 2
# Required break time 
requiredBreakTolerance = 0.05 # in hours

# Flagger

## Exceptions
ex = m[m["exception_count"] >= exception_trigger]
ex["exception_count"] = ex["exception_count"].astype("int64")
ex = ex.assign(message="Employee has "+ex["exception_count"].astype("str")+" exceptions. "+(ex["exception_count"]-(exception_trigger-1)).astype("str")+" more than allowance.")
ex = ex[["full_name","eid","date","message"]]

# Discrepancy
di = m[abs(m["potential_discrepancy"]) > discrepancy_trigger]
di = di.assign(message="Employee has "+round(di["potential_discrepancy"]).astype("int64").astype("str")+" discrepancy minutes. "+"The parameter is "+str(discrepancy_trigger)+" minutes.")
di = di[["full_name","eid","date","message"]]

# Schedule to Ontime
so = m[abs(m["schedule_to_ontime"])>sched_to_ontime_trigger]
so = so.assign(message="Employee has exceeded the " + str(sched_to_ontime_trigger) + " minute schedule to ontime trigger by " + str(metrics["schedule_to_ontime"] - sched_to_ontime_trigger) + " minutes." )
so = so[["full_name","eid","date","message"]]

# Ontime to Voxco



# Voxco to Ontime



# Ontime to Schedule



# Too many breaks

br = sl[["eid","date","ontime_duration","punch"]]
br = br[(br["punch"] == "break") | (br["punch"] == "lunch")]
b = br[(br["punch"] == "break")]
l = br[(br["punch"] == "lunch")]

l = l.groupby(["eid","date"],as_index=False).agg({"ontime_duration":sum,
                                                  "punch":np.size})
ll = l[l["ontime_duration"] > lunch_length]
lc = l[l["punch"] > lunch_count]
ll = ll.assign(message="Lunch too long: " + ll["ontime_duration"].astype("str") + " minutes.")
ll = ll[["eid","date","message"]]
lc = lc.assign(message="Too many lunches: " + lc["punch"].astype("str") + " lunch slices.")
lc = lc[["eid","date","message"]]

b = b.groupby(["eid","date"],as_index=False).agg({"ontime_duration":sum,
                                                  "punch":np.size})
bl = b[b["ontime_duration"] > break_length]
bc = b[b["punch"] > break_count]
bl = bl.assign(message="break too long: " + bl["ontime_duration"].astype("str") + " minutes.")
bl = bl[["eid","date","message"]]
bc = bc.assign(message="Too many breaks: " + bc["punch"].astype("str") + " break slices.")
bc = bc[["eid","date","message"]]

breaks = pd.concat([ll,lc,bl,bc])
breaks = pd.merge(breaks,employee,on=["eid"],how="left")
breaks = breaks[(breaks["Division"]=="CRD")&(breaks["Job Title"]=="Research Associate")]
breaks = breaks[["full_name","eid","date","message"]]

# Schedule outside of availability

soa = m[(m["sched_start"]<m["avail_start"]) | (m["avail_end"]<m["avail_start"])]
soa = soa.assign(message="RAs schedule does not fall within availability parameters.")
soa = soa[["full_name","eid","date","message"]]

# Missed Required Break
mb = variance[variance["status"]=="ACTIVE"]
mb = mb.assign(miss=mb["actual-break"]<mb["paid-break"]-requiredBreakTolerance)
mb = mb[mb["miss"]==True]
mb = mb.rename(columns={"EID":"eid"})
mb = pd.merge(mb,employee,on="eid",how="left")
mb = mb.assign(message="Employee has not taken enough breaks by law.")
mb = mb[["full_name","eid","date","message"]]
# Concerns
concerns = pd.concat([di,ex,so,soa,mb])
data_table.DataTable(concerns)


# Punch List

In [ ]:
# Voxco punches
vcPunches = sesh[["eid","vc_project","date","vc_login","vc_logout"]]
vcPunches = vcPunches.assign(system="voxco")
vcPunches["date"] = pd.to_datetime(vcPunches["date"])
vcPunches = vcPunches.rename(columns={"vc_login":"in",
                                      "vc_logout":"out",
                                      "vc_project":"project"})
# OnTime punches
otPunches = sl[["eid","ontime_project","date","ontime_start","ontime_end", "punch"]]
otPunches = otPunches.assign(system="ontime")
otPunches["date"] = pd.to_datetime(otPunches["date"])
otPunches = otPunches.rename(columns={"ontime_start":"in",
                                      "ontime_end":"out",
                                      "ontime_project":"project",
                                      "punch":"ontime_punch"})
# Union
punchList = pd.concat([vcPunches,otPunches])
punchList = punchList.sort_values(by=["eid","in"])
# Sort
punchList = punchList[punchList["eid"]!=0]
punchList = punchList.sort_values(by = ["eid","in"])
# name
punchList = pd.merge(punchList,employee,on="eid",how="left")
punchList = punchList[["eid","full_name","project","date","in","out","system","ontime_punch"]]

#punchList = punchList[punchList["date"] == date.today() -  timedelta(days=1)]
#diag(vcPunches,None,None)
#data_table.DataTable(punchList)

In [ ]:
# Attendance
att = variance[["EID","name","date","department","status","scheduled-work","actual-work","schedule-status"]]
att = att[(att["status"]=="ACTIVE")]
att = att.assign(workRate=att["actual-work"] / att["scheduled-work"])
att["workRate"] = round(att["workRate"].astype("float64"),2)
att = att.assign(alias=att["name"] + " - " + att["EID"].astype("str"))
att["baz"] = pd.DataFrame.where(cond=att["schedule-status"]=="NORMAL", self= att["workRate"].astype("str"), other= att["schedule-status"])
att["date"] = pd.to_datetime(att["date"])
realRate = att[att["schedule-status"]== "NORMAL"]

realRate = realRate.groupby(["alias"], as_index = False).agg({"workRate": np.mean})

ncns = att[(att["schedule-status"]== "NORMAL")&(att["workRate"]==0.0)]
ncns = ncns.groupby(["alias"], as_index =False).size()
ncns = ncns.rename(columns={"size":"ncns"})

atp = att.pivot(index="alias",columns="date",values="baz")
atp = pd.merge(atp,realRate,on="alias",how="left")
atp = pd.merge(atp,ncns,on="alias",how="left")

atp["workRate"] = round(atp["workRate"],2)
atp["ncns"] = atp["ncns"].fillna(0)
atp["ncns"] = atp["ncns"].astype("int64")
atp = atp.astype("str")
#data_table.DataTable(atp)
#print(atp.dtypes)
#print(atp.head(50))





In [ ]:
# Attendance Version 2

at = variance[["EID","name","date","department","status","scheduled-work","actual-work","schedule-status"]]
# make a table with eid and job title so that you can 
# filter where equals "Research Associate"
e = employee[["eid","Job Title"]]
e = e.rename(columns={"eid":"EID"})
at = pd.merge(at,e,on="EID",how="left")
at = at[(at["Job Title"]=="Research Associate")]

at = at[(at["status"]=="ACTIVE")]
at["date"] = pd.to_datetime(at["date"])
at = at.assign(rate=at["actual-work"] / at["scheduled-work"])
at["rate"] = round(at["rate"],2)
at = at.assign(alias=at["name"] + " - " + at["EID"].astype("str"))
at["baz"] = pd.DataFrame.where(cond=at["schedule-status"]=="NORMAL", self= at["rate"].astype("str"), other= at["schedule-status"])
piv = at.pivot(index="alias",columns="date",values ="baz")
# JA process people who have not shown up for three days
ja = variance[variance["status"]=="ACTIVE"]
ja["date"] = pd.to_datetime(ja["date"])
ja = ja.assign(ranker=ja.groupby(["EID"])["date"].rank(method="first", ascending=False))
ja = ja[(
          ((ja["schedule-status"]=="NORMAL")&(ja["actual-work"]==0.0)&(ja["ranker"]==1.0)) |
          ((ja["schedule-status"]=="NORMAL")&(ja["actual-work"]==0.0)&(ja["ranker"]==2.0)) |
          ((ja["schedule-status"]=="NORMAL")&(ja["actual-work"]==0.0)&(ja["ranker"]==3.0))
       )]
ja = ja.groupby(["EID","name"],as_index=False).agg({"ranker":sum})
ja = ja[ja["ranker"]==6.0]
ja = ja.assign(ja="ja flag")
ja = ja.assign(alias=ja["name"] + " - " + ja["EID"].astype("str"))
ja = ja[["alias","ja"]]
ja = ja.drop_duplicates()
# Check that the person has been scheduled for at least 23 hours in the past week
# Try to find an excuse?
# Run three weeks worth of work rate
s = sched.rename(columns={"eid":"EID"})
variance["date"] = pd.to_datetime(variance["date"])

d = pd.merge(variance,s,on=["EID","date"],how="left")

d = d.assign(schedLen=d["sched_end"]-d["sched_start"])
d["schedLen"] = d["schedLen"].dt.total_seconds()
d["schedLen"] = round(d["schedLen"] / 60 / 60,2)
d = d.assign(alias=d["name"]+" - "+d["EID"].astype("str"))

ds = d[d["date"]>= pd.to_datetime(date.today()-timedelta(days=7))]

ds = ds.groupby(["alias"],as_index=False).agg({"schedLen":sum})

d = d.groupby(["alias"],as_index=False).agg({"actual-work":sum,
                                              "scheduled-work":sum})
d = d.assign(workRate=d["actual-work"] / d["scheduled-work"])
d["workRate"] = round(d["workRate"],2)

d = d[["alias","workRate"]]
ds = ds[["alias","schedLen"]]
piv = pd.merge(piv,ja,on="alias",how="left")
piv = pd.merge(piv,d,on="alias",how="left")
piv = pd.merge(piv,ds,on="alias",how="left")
piv = piv.fillna("")


# Requested absences
#a = at[["EID","alias"]]
##a = a.rename(columns = {"EID":"eid"})
#r = pd.merge(avail,a,on="eid",how="left")
#requests = requests.rename(columns={"detail.date":"date",
#                                    "detail.associate":"eid"})
#requests["date"] = pd.to_datetime(requests["date"])
#r = pd.merge(r,requests,on="eid",how="right").dropna().drop_duplicates()



print(piv)




In [ ]:
# Resource Allocation
ra = avail.assign(hrs=avail["avail_end"] - avail["avail_start"])
ra["hrs"] = round(ra["hrs"].dt.total_seconds() / 60 / 60,2)
def parse_values(x):
    if x >= 8.5:
       return 8.5
    else:
       return x
ra["hrs"] = ra["hrs"].apply(parse_values)
ra = pd.merge(ra,sched,on=["eid","date"],how="left")
ra = ra.assign(shrs=ra["sched_end"] - ra["sched_start"])
ra["shrs"] = round(ra["shrs"].dt.total_seconds() / 60 / 60,2)

tb = ra[["date","eid","sched_project","hrs","shrs"]]

ra["hrs"] = ra["hrs"].fillna(0.0)
ra["shrs"] = ra["shrs"].fillna(0.0)

ra = pd.merge(ra,employee,on="eid",how="left")
ra = ra[(ra["Status"]=="Active")&(ra["Job Title"]=="Research Associate")]
ra = ra.rename(columns={"hrs":"available hours",
                        "shrs":"scheduled hours"})
ra = ra.assign(unscheduled=ra["available hours"] - ra["scheduled hours"])
ra = ra[["date","eid","full_name","sched_project","available hours","scheduled hours","unscheduled"]]


data_table.DataTable(ra)

In [ ]:
# CRD Stats
# total time in ontime versus total time in Voxco, discrepancy
# Monitoring sessions count and score
# call count, complete count

# Employee Table
#em = employee[employee["Job Title"]=="Research Associate"]
# Availability Table - total hours available
#av = avail.assign(avail_dur=avail["avail_end"] - avail["avail_start"])
#av["avail_dur"] = av["avail_dur"].dt.total_seconds() / 60 / 60
# Exception Table
#ex = excep.groupby(["date","eid"],as_index=False).agg({"exception_count":sum})
# Score Table
#sc = scores.groupby(["date","eid"],as_index=False).agg({"score":np.mean,
#                                                        "cases":np.size})
#sc["date"] = pd.to_datetime(sc["date"])
# Variance Table
#vr = variance[["EID","date","scheduled-work","actual-work","paid-break","actual-break","late-minutes","schedule-status"]]
#vr = vr.rename(columns={"EID":"eid"})
# Schedule Table
#scd = sched[["eid","date","sched_start","sched_end"]]
# Sessions Table
#sessions["date"]  = pd.to_datetime(sessions["LOGIN_DATE"])
#sessions = sessions[(sessions["bamboo_id"]!="kbigelow")&
#                    (sessions["bamboo_id"]!="26045B")&
#                    (sessions["bamboo_id"]!="tstrauss")]

#sessions["eid"] = sessions["bamboo_id"] 
#se = sessions.groupby(["date","eid"],as_index=False).agg({"sesh_dur":sum,
#                                                          "talkTime":sum,
#                                                          "waitingTime":sum,
#                                                          "pauseTime":sum,
#                                                          "reviewTime":sum,
#                                                          "soat":sum)

# Everything together
#stats = pd.merge(em,av,on="eid",how="left")
#stats["date"] = pd.to_datetime(stats["date"])
#stats = pd.merge(stats,ex,on=["eid","date"],how="left")
#stats = pd.merge(stats,sc,on=["eid","date"],how="left")
#stats = pd.merge(stats,vr,on=["eid","date"],how="left")
#stats = pd.merge(stats,scd,on=["eid","date"],how="left")
#se["eid"] = se["eid"].astype("int64")
#stats = pd.merge(stats,se,on=["date","eid"],how="left")

# Varibles
# Week ago
#seven_days = date.today()-timedelta(days=7)

# New Columns
#df['new column name'] = df['column name'].apply(lambda x: 'value if condition is met' if x condition else 'value if condition is not met')
#stats["new_hire"] = stats["hire_date"].apply(lambda x: "yes" if x >= seven_days else "no")
#stats = stats.assign(sched_len=stats["sched_end"] - stats["sched_start"])
#stats["sched_len"] = stats["sched_len"].dt.total_seconds() / 60 / 60

# Aggregation

#e = stats.groupby(["full_name","eid","Status","Division","Job Title","new_hire"])

#print(stats[stats["eid"]==65])
#data_table.DataTable(stats)

In [ ]:
proj = project.groupby(["EID","project","date"],as_index=False).agg({"hours":sum})
proj = proj.rename(columns={"EID":"eid"})
proj["hours"] = round(proj["hours"],2)
vox = sesh.groupby(["eid","vc_project","date"],as_index=False).agg({"sesh_dur":sum})
vox = vox.rename(columns={"vc_project":"project"})
vox["sesh_dur"] = round(vox["sesh_dur"] / 60 / 60,2)
vox["project"] = vox["project"].str.lower().str.replace("_","-")
vox["date"] = pd.to_datetime(vox["date"])
ov = pd.merge(proj,vox,on=["eid","date","project"],how="left")
ov = ov.rename(columns = {"hours":"ontime",
                          "sesh_dur":"voxco"})
ov = ov.assign(diff=ov["ontime"] - ov["voxco"])
ov["diff"] = round(ov["diff"],2)
ee = employee[["eid","full_name","Job Title"]]
ov = pd.merge(ov,ee,on="eid",how="left")
ov["projecta"] = ov["project"].str.contains("cf",case=False,regex=False)
ov = ov[ov["projecta"]==True]
ov = ov[ov["Job Title"]=="Research Associate"]

ov = ov.sort_values(by=["full_name","date"])
ov = ov[["eid","full_name","date","project","ontime","voxco","diff"]]
data_table.DataTable(ov)

# Downloads

In [ ]:
# Performance manager report
from datetime import datetime, timedelta
#if date.today().weekday() < 6:
#  punchList = punchList[punchList["date"]== datetime.today() - timedelta(days=1)]
#  flags = flags[flags["date"]== datetime.today() - timedelta(days=1)]
#else:
#  punchList = punchList[punchList["date"]>= datetime.today() - timedelta(days=3)]
#  flags = flags[flags["date"]>= datetime.today() - timedelta(days=3)]

flags = concerns.drop_duplicates() #pd.concat([di, breaks])
with pd.ExcelWriter('CF_Performance_Manager_Report '+ date.today().strftime("%d-%b-%Y") +'.xlsx',
                    datetime_format = 'YYYY-MM-DD HH:MM:SS') as writer:
                    punchList.to_excel(writer, sheet_name = 'Punch List', index=False)
                    flags.to_excel(writer, sheet_name = 'Flags', index=False)
                    piv.to_excel(writer, sheet_name = 'Attendance', index=False)
                    ra.to_excel(writer, sheet_name = 'schedule projection', index=False)
                    #stats.to_excel(writer, sheet_name = 'Statistics', index=False)
                    ov.to_excel(writer, sheet_name = "discrepancy", index = False)
files.download('CF_Performance_Manager_Report '+ date.today().strftime("%d-%b-%Y") +'.xlsx')

# Data Visualizations